This file takes in colin's data.zip from 05/03 and adds hyunwoo's fingerprint.

In [6]:
import glob, numpy as np, os, shutil, pickle, rdkit, sys, tqdm, fingerprint_utils, torch

assert(os.path.exists("../../tempdata/data"))

Creating hyun_fp_data

In [5]:
def shatter(shatter_dir, obj, id, keys):
    '''
        Saves an object as individual files
    '''
    for key in keys:
        elem = obj[key]
        if type(elem) == np.ndarray:
            torch.save(torch.tensor(elem, dtype=torch.float), f"{shatter_dir}/{key}/{id}.pt")
        elif type(elem) == str:
            torch.save(elem, f"{shatter_dir}/{key}/{id}.pt")


In [ ]:
files = glob.glob("../../tempdata/data/*/*.pkl")
out_dir = "../../tempdata/hyun_fp_data"
os.makedirs(out_dir, exist_ok=True)
save_attrs = {
    "hsqc_ms_pairs": ["SMILES", "FP", "HYUN_FP", "HSQC", "MS"],
    "hsqc_pretrain": ["SMILES", "FP", "HYUN_FP", "HSQC"],
    "ms_pretrain": ["SMILES", "FP", "HYUN_FP", "MS"]
}
for file_path in files:
    subdivision = file_path.split("/")[-2]
    split = file_path.split("/")[-1]
    split = split[:split.index(".")]

    for my_attr in save_attrs[subdivision]:
        os.makedirs(f"{out_dir}/{subdivision}/{split}/{my_attr}", exist_ok=True)

    print(file_path, subdivision, split)
    with open(file_path, "rb") as f:
        obj = pickle.load(f)

    keys = set(obj.keys())
    for k in keys:
        v = obj[k]
        v["HYUN_FP"] = fingerprint_utils.FP_generator(v["SMILES"], 2)
        shatter(f"{out_dir}/{subdivision}/{split}", v, k, save_attrs[subdivision])
        del obj[k]

In [ ]:
for f in os.listdir(out_dir):
    for g in os.listdir(os.path.join(out_dir, f)):
        print(f"{f}/{g}/{len(os.listdir(os.path.join(out_dir, f, g, 'FP')))}")

New Dataset using better bounds / hsqc intensity rounding

In [11]:
v = torch.randint(-10, 10, (30, 50, 3)).float()
v[:,:,2] = torch.where(v[:,:,2] >= 0, 1.0, -1.0)

tensor([[  6.,  -4.,  -8.,  ...,  -7.,  -7.,  -6.],
        [ -6.,   1.,   9.,  ...,  -8.,  -8.,  -6.],
        [  3., -10., -10.,  ...,  -6.,   1.,   9.],
        ...,
        [ -2.,   3.,  -6.,  ...,   4.,   9.,   7.],
        [ -1.,   8.,  -6.,  ...,   7., -10.,  -7.],
        [ -7.,   8.,  -2.,  ...,   6.,  -6.,   3.]])
tensor([[ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        ...,
        [-1.,  1., -1.,  ...,  1.,  1.,  1.],
        [-1.,  1., -1.,  ...,  1., -1., -1.],
        [-1.,  1., -1.,  ...,  1., -1.,  1.]])


In [26]:
import numpy as np
def shatter2(shatter_dir, obj, id, keys):
    '''
        Saves an object as individual files
    '''
    for key in keys:
        elem = obj[key]
        if type(elem) == np.ndarray:
            if key == "HSQC":
                elem[:,2] = np.where(elem[:,2] >= 0, 1.0, -1.0)
            torch.save(torch.tensor(elem, dtype=torch.float), f"{shatter_dir}/{key}/{id}.pt")
        elif type(elem) == str:
            torch.save(elem, f"{shatter_dir}/{key}/{id}.pt")


In [27]:
files = glob.glob("../../tempdata/data/*/*.pkl")
out_dir = "../../tempdata/bounded_hyun_fp_data"
os.makedirs(out_dir, exist_ok=True)

save_attrs = {
    "hsqc_ms_pairs": ["SMILES", "FP", "HYUN_FP", "HSQC", "MS"],
    "hsqc_pretrain": ["SMILES", "FP", "HYUN_FP", "HSQC"],
    "ms_pretrain": ["SMILES", "FP", "HYUN_FP", "MS"]
}

for file_path in files:
    subdivision = file_path.split("/")[-2]
    split = file_path.split("/")[-1]
    split = split[:split.index(".")]

    for my_attr in save_attrs[subdivision]:
        os.makedirs(f"{out_dir}/{subdivision}/{split}/{my_attr}", exist_ok=True)

    print(file_path, subdivision, split)
    with open(file_path, "rb") as f:
        obj = pickle.load(f)

    keys = set(obj.keys())
    print(save_attrs[subdivision])
    for k in keys:
        v = obj[k]
        v["HYUN_FP"] = fingerprint_utils.FP_generator(v["SMILES"], 2)
        shatter2(f"{out_dir}/{subdivision}/{split}", v, k, save_attrs[subdivision])
        del obj[k]

../../tempdata/data/hsqc_ms_pairs/train.pkl hsqc_ms_pairs train
['SMILES', 'FP', 'HYUN_FP', 'HSQC', 'MS']


[03:29:51] WARNING: not removing hydrogen atom without neighbors


../../tempdata/data/hsqc_ms_pairs/val.pkl hsqc_ms_pairs val
['SMILES', 'FP', 'HYUN_FP', 'HSQC', 'MS']
../../tempdata/data/hsqc_ms_pairs/test.pkl hsqc_ms_pairs test
['SMILES', 'FP', 'HYUN_FP', 'HSQC', 'MS']
../../tempdata/data/hsqc_pretrain/train.pkl hsqc_pretrain train
['SMILES', 'FP', 'HYUN_FP', 'HSQC']


[03:31:20] WARNING: not removing hydrogen atom without neighbors
[03:31:28] WARNING: not removing hydrogen atom without neighbors
[03:31:32] WARNING: not removing hydrogen atom without neighbors
[03:31:55] WARNING: not removing hydrogen atom without neighbors
[03:32:06] WARNING: not removing hydrogen atom without neighbors
[03:33:18] WARNING: not removing hydrogen atom without neighbors
[03:34:02] WARNING: not removing hydrogen atom without neighbors
[03:35:41] WARNING: not removing hydrogen atom without neighbors
[03:36:27] WARNING: not removing hydrogen atom without neighbors
[03:37:54] WARNING: not removing hydrogen atom without neighbors


../../tempdata/data/hsqc_pretrain/val.pkl hsqc_pretrain val
['SMILES', 'FP', 'HYUN_FP', 'HSQC']


[03:39:11] WARNING: not removing hydrogen atom without neighbors
[03:39:30] WARNING: not removing hydrogen atom without neighbors


../../tempdata/data/hsqc_pretrain/test.pkl hsqc_pretrain test
['SMILES', 'FP', 'HYUN_FP', 'HSQC']


[03:40:11] WARNING: not removing hydrogen atom without neighbors
[03:40:11] WARNING: not removing hydrogen atom without neighbors


../../tempdata/data/ms_pretrain/train.pkl ms_pretrain train
['SMILES', 'FP', 'HYUN_FP', 'MS']
../../tempdata/data/ms_pretrain/val.pkl ms_pretrain val
['SMILES', 'FP', 'HYUN_FP', 'MS']
../../tempdata/data/ms_pretrain/test.pkl ms_pretrain test
['SMILES', 'FP', 'HYUN_FP', 'MS']


In [15]:
for f in os.listdir(out_dir):
    for g in os.listdir(os.path.join(out_dir, f)):
        print(f"{f}/{g}/{len(os.listdir(os.path.join(out_dir, f, g, 'FP')))}")

hsqc_ms_pairs/train/1


Creating same dataset, but using hsqc as images

In [ ]:
# @wangdong make sure you pull colin_split_05_23_22.zip before running this code
# this creates the pickle files into ../../tempdata/data/...
def shatter(shatter_dir, obj, id, keys):
    '''
        Saves an object as individual files
    '''
    for key in keys:
        elem = obj[key]
        if type(elem) == np.ndarray:
            if key == "HSQC":
                # @wangdong you can add handler functions here to pre-process your data
                elem2 = elem
                torch.save(elem2, f"{shatter_dir}/{key}/{id}.pt")
                pass
            else:
                torch.save(torch.tensor(elem, dtype=torch.float), f"{shatter_dir}/{key}/{id}.pt")
        elif type(elem) == str:
            torch.save(elem, f"{shatter_dir}/{key}/{id}.pt")

files = glob.glob("../../tempdata/data/*/*.pkl") # colin's input
output_dir_name = None # @wandong change the output directory here
assert(output_dir_name)
out_dir = f"../../tempdata/{output_dir_name}"
os.makedirs(out_dir, exist_ok=True)
save_attrs = {
    "hsqc_ms_pairs": ["SMILES", "FP", "HYUN_FP", "HSQC", "MS"],
    "hsqc_pretrain": ["SMILES", "FP", "HYUN_FP", "HSQC"],
    "ms_pretrain": ["SMILES", "FP", "HYUN_FP", "MS"]
}
for file_path in files:
    subdivision = file_path.split("/")[-2]
    split = file_path.split("/")[-1]
    split = split[:split.index(".")]

    for my_attr in save_attrs[subdivision]:
        os.makedirs(f"{out_dir}/{subdivision}/{split}/{my_attr}", exist_ok=True)

    print(file_path, subdivision, split)
    with open(file_path, "rb") as f:
        obj = pickle.load(f)

    keys = set(obj.keys())
    for k in keys:
        v = obj[k] # the object that you want to split into directories
        v["HYUN_FP"] = fingerprint_utils.FP_generator(v["SMILES"], 2)
        shatter(f"{out_dir}/{subdivision}/{split}", v, k, save_attrs[subdivision])
        del obj[k]